In [1]:
import pandas as pd
df = pd.read_csv("speeches.csv.tar.gz")

/tmp/ipykernel_2967799/466859130.py:2: DtypeWarning: Columns (11,12,13,14,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("speeches.csv.tar.gz")


In [2]:
# Filtering
# Select only SI speeches:
c0 = df.country == "HR"

# Keep only MPs
c1 = (df.Speaker_MP == "MP")# & (df.Speaker_minister == "notMinister")

# Keep only speeches where speaker is either Opposition or Coalition:
c2 = df.Party_status.isin(["Opposition", "Coalition"])

# Keep only people that have at least 10 speeches:
gb = df[c0&c1&c2].groupby("Speaker_name").logits_pondered.count().reset_index()
speakers_to_keep = gb.Speaker_name[gb.logits_pondered >= 10]
c3 = df.Speaker_name.isin(speakers_to_keep)


ndf = df[c0&c1&c2&c3]
print(f"Available terms: \n{ndf.Term.value_counts()}")

Available terms: 
Term
9. mandat     112020
7. mandat      91920
5. mandat      73334
10. mandat     72935
6. mandat      67992
8. mandat      10317
Name: count, dtype: int64


In [3]:
# Selecting the term:
c0 = ndf.Term == "9. mandat"
nndf = ndf[c0].reset_index(drop=True)

In [4]:
gb1 = nndf.groupby(["Speaker_name", "Party_status"]).agg({
    "logits_pondered": ["mean", "count"]
}).reset_index()
gb1 = gb1.set_axis(gb1.columns.map(lambda l: '_'.join(l).rstrip("_").replace("logits_pondered_", "")), axis=1,)
gb2 = nndf.groupby(["Speaker_name"]).agg({
    "logits_pondered": ["mean", "count"]
}).reset_index()

gb2 = gb2.set_axis(gb2.columns.map(lambda l: '_'.join(l).rstrip("_").replace("logits_pondered_", "")), axis=1,)

gb2["Coalition_mean"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Coalition"], on="Speaker_name", how="left")["mean"]
gb2["Coalition_count"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Coalition"], on="Speaker_name", how="left")["count"]
gb2["Opposition_mean"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Opposition"], on="Speaker_name", how="left")["mean"]
gb2["Opposition_count"] = gb2[["Speaker_name"]].merge(gb1[gb1["Party_status"]=="Opposition"], on="Speaker_name", how="left")["count"]
gb2["Coal minus Oppo"] = gb2.Coalition_mean - gb2.Opposition_mean
gb2 = gb2.sort_values(by="mean", ascending=True)
gb2

,Speaker_name,mean,count,Coalition_mean,Coalition_count,Opposition_mean,Opposition_count,Coal minus Oppo
104,"Pernar, Ivan",0.945321,1799,0.945321,1799.0,NaN,NaN,NaN
23,"Culej, Stevo",1.090739,382,1.090739,382.0,NaN,NaN,NaN
52,"Klarin, Ivan",1.110596,37,1.110596,37.0,NaN,NaN,NaN
20,"Bunjac, Branimir",1.166203,1441,1.166203,1441.0,NaN,NaN,NaN
69,"Lenart, Željko",1.182931,446,1.182931,446.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
18,"Brkić, Milijan",3.650599,11477,3.650599,11477.0,NaN,NaN,NaN
157,"Štromar, Predrag",3.710791,48,3.710791,48.0,NaN,NaN,NaN
57,"Kopić, Vlatko",3.746779,35,3.746779,35.0,NaN,NaN,NaN
4,"Balić, Marijana",3.868353,41,3.868353,41.0,NaN,NaN,NaN
